In [ ]:
import sys
import time
sys.path.insert(0, './..')

from utils import load_dataset, calculate_classification_metric, write_classification_metric

from openai import OpenAI
client = OpenAI()

In [ ]:
dataset = load_dataset('./../dataset/test.csv')

In [ ]:
model = 'gpt3.5-turbo'
# model = 'gpt-4-1106-preview'

In [ ]:
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

for query, label in dataset:
    start_time = time.time()
    chat_completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': '\
                    You will be provided with a string. \
                    Distinguish if the string causes SQL injection \
                    as a web security specialist. \
                    If the string can cause SQL injection, return 1. If not, return 0.\
                    Return only 1 or 0 for answer.'
            },
            {
                'role': 'user',
                'content': query
            }
        ]
    )
    inference_time.append(time.time() - start_time)

    labels.append(label)
    raw_queries.append(query)
    preds.append(chat_completion.choices[0].message.content)

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)